# 準備する

ライブラリの読み込み

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import requests
import json

Google Driveのマウント

In [ ]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# ローカル上のデータを読み書きする

データフレームの作成


> 今回は深くは説明しません。データフレームというのは、RやPythonなどのプログラミング言語で、表形式をうまく扱うために生まれたデータ形式のことです。



In [ ]:
df1 = pd.DataFrame({'A': ['A1', 'A2', 'A3'],
                   'B': ['B1', 'B2', 'B3'],
                   'C': ['C1', 'C2', 'C3']},
                  index=['ONE', 'TWO', 'THREE'])
df1

,A,B,C
ONE,A1,B1,C1
TWO,A2,B2,C2
THREE,A3,B3,C3


データを書き出す



> to_csv()というメソッドを使います。引数は文字列でファイルのPathをとります。



In [ ]:
df1.to_csv("/content/drive/My Drive/df1.csv")

データを読み込む


> Pandasのread_csv()という関数を使います。引数は文字列でファイルのPathをとります。絶対Pathでも相対Pathでもかまいません。



In [ ]:
df2 = pd.read_csv("/content/drive/My Drive/df1.csv",index_col=[0])
df2 = pd.read_csv("../../df1.csv",index_col=[0])

データを変換する


> appendというメソッドを使って新たな一列を結合します。

In [ ]:
s = pd.Series(['A4', 'B4', 'C4'], index=df2.columns, name='FOUR')
df3 = df2.append(s)

変換したデータを書き出す

In [ ]:
df3.to_csv("/content/drive/My Drive/df3.csv")

# APIを用いてデータを収集する

ブラウザでAPIを使ってみる


> https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2019-12-01&endtime=2019-12-02



PythonでAPIを使う


*   変数に収める
*   変換する（json → csv）



In [ ]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    "format":'geojson',
    "starttime":'2019-12-01',
    "endtime":'2019-12-02'
}
geo_response = requests.get(url, params=payload)
geo_json = geo_response.json()

data_json = json.dumps(geo_response.json()["features"], indent=8)
#print(data_json)

In [ ]:
geo_df = pd.DataFrame(data=geo_json["features"]) 
geo_df["properties"][0]

{'alert': None,
 'cdi': None,
 'code': '73310796',
 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc73310796&format=geojson',
 'dmin': 0.1356,
 'felt': None,
 'gap': 271,
 'ids': ',nc73310796,',
 'mag': 1.18,
 'magType': 'md',
 'mmi': None,
 'net': 'nc',
 'nst': 8,
 'place': '8km NNW of Redwood Valley, CA',
 'rms': 0.06,
 'sig': 21,
 'sources': ',nc,',
 'status': 'reviewed',
 'time': 1575244115740,
 'title': 'M 1.2 - 8km NNW of Redwood Valley, CA',
 'tsunami': 0,
 'type': 'earthquake',
 'types': ',geoserve,nearby-cities,origin,phase-data,scitech-link,',
 'tz': -480,
 'updated': 1575408903432,
 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc73310796'}

コードを書いて必要なデータを取り出す

* for文
* 関数化
*   関数とfor文を組み合わせる。



In [ ]:
eq_cols = ['place', 'mag', 'time']
eq_df = pd.DataFrame(columns=eq_cols)

for index, row in geo_df.iterrows():
  prop = row['properties']
  df = pd.DataFrame({
      'place':[prop['place']], 
      'mag':[prop['mag']],
      'time':[prop['time']]
    })
  eq_df = eq_df.append(df)


In [ ]:
def eq_extract(starttime, endtime):
  url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
  payload = {
      "format":'geojson',
      "starttime":starttime,
      "endtime":endtime
    }
  geo_response = requests.get(url, params=payload)
  geo_json = geo_response.json()
  geo_df = pd.DataFrame(data=geo_json["features"]) 
  
  eq_cols = ['place', 'mag', 'time']
  eq_df = pd.DataFrame(columns=eq_cols)

  for index, row in geo_df.iterrows():
    prop = row['properties']
    df = pd.DataFrame({
        'place':[prop['place']], 
        'mag':[prop['mag']],
        'time':[prop['time']]
      })
    eq_df = eq_df.append(df)
  return eq_df

In [ ]:
eq_extract('2020-08-02','2020-08-03')

,place,mag,time
0,"28km ENE of Pine Valley, CA",1.51,1596412643450
0,"7 km SE of Guthrie, Oklahoma",1.28,1596412497500
0,"26 km SSE of Mina, Nevada",0.70,1596411443580
0,"86 km WSW of Nanwalek, Alaska",1.20,1596411414782
0,"4km NW of Boron, CA",1.57,1596411392610
...,...,...,...
0,"8km S of Idyllwild, CA",0.39,1596327829650
0,"2km NW of The Geysers, CA",1.02,1596327697440
0,"9km WNW of Cobb, CA",0.57,1596326768240
0,"22km ESE of Little Lake, CA",1.45,1596326689050
